<a href="https://colab.research.google.com/github/nancyfosua/NLP_Assignment/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [4]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [5]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]
        
        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [6]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

In [9]:
word_dict, label_dict = build_dict("/content/drive/MyDrive/Colab Notebooks/NLP/week1/train1.txt")

In [23]:
train_data = load_data("/content/drive/MyDrive/Colab Notebooks/NLP/week1/train1.txt", word_dict, label_dict)
train_data[3]

(0, array([0., 0., 0., ..., 0., 0., 0.]))

In [27]:
 y= train_data[1][0]
 y

0

First, let's implement the softmax function. Don't forget numerical stability!

In [1]:
def softmax(x):
    ### FILL CODE
    s = np.exp(x-x.max())/ (np.sum(np.exp(x-x.max())))
    return s

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [99]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    print(nlabels, dim)
    print(len(data))
    for iter in range(niter):
        ### FILL CODE
      for y, x in data:
        #make prediction
        y_pred = softmax(np.dot(w,x))

        #calculate the loss
        trg = np.zeros(nlabels)
        trg[y] = 1
        loss = y_pred - trg
        # calculate the gradient
        grad = loss.reshape((nlabels,1)) * x.reshape((1, dim))  #x.T @ (x @ w - y)
        #update the gradient
        w = w - 0.1 * grad
      print("epoch number ", iter)
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [106]:
def predict(w, x):
    ## FILL CODE
    yPred = np.dot(w,x)
    return np.argmax(softmax(yPred)) 

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [107]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    accuracy = 0
    for data in valid_data:
      pred = predict(w, data[1])
      if data[0] == pred :
        accuracy +=1
    return accuracy/len(valid_data)

In [108]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("/content/drive/MyDrive/Colab Notebooks/NLP/week1/train1.txt")
train_data = load_data("/content/drive/MyDrive/Colab Notebooks/NLP/week1/train1.txt", word_dict, label_dict)
valid_data = load_data("/content/drive/MyDrive/Colab Notebooks/NLP/week1/valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **

10 5826
10000
epoch number  0
epoch number  1
epoch number  2
epoch number  3
epoch number  4

Validation accuracy: 0.923



In [91]:
valid_data[0]

(4, array([0., 0., 0., ..., 0., 0., 0.]))